In [1]:
import sys
sys.path.extend(["../../../../"])

In [2]:
import numpy as np
from onsager.crystal import Crystal
from onsager.crystalStars import zeroclean
from onsager.OnsagerCalc import *
from onsager.crystal import DB_disp, DB_disp4, pureDBContainer, mixedDBContainer
from onsager.DB_structs import dumbbell, SdPair, jump, connector

from scipy.constants import physical_constants
kB = physical_constants['Boltzmann constant in eV/K'][0]
from matplotlib import pyplot as plt
from collections import defaultdict

import pickle

In [3]:
%%time
# Let's load the pickle file we just saved
with open("FeX_60RT.pkl","rb") as fl:
    onsagercalculator = pickle.load(fl)

CPU times: user 8min 30s, sys: 5.65 s, total: 8min 36s
Wall time: 8min 39s


In [4]:
# Next, we assign labels to the states we have identified in the Fe-Cr notebook
name_to_themo_star = {"1nnA":2, "1nnB":1, "2nnA":4, "2nnB":3, "3nnA": 7, "3nnB":5, "3nnC":6,
       "4nnA":11,"4nnB":10,"4nnC":9, "4nnD":8, "5nnA":13, "5nnB":12}

In [5]:
# sorting out the jumps with the nomenclatures
jmpdict = {"1nnA_2nnA":[], "1nnA_2nnB":[], "1nnA_3nnB":[], "1nnA_3nnC":[], "1nnB_2nnB":[], "1nnB_3nnB":[],
          "1nnB_5nnB":[], "2nnA_4nnC":[], "2nnB_4nnB":[], "2nnB_4nnC":[]}
# Now identify the jumps and put them into the dictionaries
for jlistind, jlist in enumerate(onsagercalculator.jnet1):
    jmp = jlist[0]
    state1 = jmp.state1
    state2 = jmp.state2
    # if rigid jump, then continue
    if jmp.state1.db.iorind == jmp.state2.db.iorind:
        continue
    star1 = onsagercalculator.kinetic.complexIndexdict[state1][1]
    star2 = onsagercalculator.kinetic.complexIndexdict[state2][1]
    
    if star1 in onsagercalculator.thermo2kin and star2 in onsagercalculator.thermo2kin:
        thermo_star1 = onsagercalculator.thermo.complexIndexdict[state1][1]
        thermo_star2 = onsagercalculator.thermo.complexIndexdict[state2][1]
        name1 = ""
        name2 = ""
        #Now see which categories the states belong to
        star1found = False
        count1 = 0
        star2found = False
        count2 = 0
        for (key, value) in name_to_themo_star.items():
            if thermo_star1==value:
                star1found = True
                count1 += 1
                name1 = key
            if thermo_star2==value:
                star2found = True
                count2 += 1
                name2 = key
        # just to ensure we don't have any multiple counting business going on.
        if count1>1:
            print(thermo_star1)
        if count2>1:
            print(thermo_star2)
        # Now concatenate names
        jname = name1+"_"+name2
        jnameRev = name2+"_"+name1
        try:
            jmpdict[jname].append(jlistind)
        except:
            try:
                # maybe the jump we have is the reverse of what we stored as the label in the dictionary?
                jmpdict[jnamerev].append(jlistind)
            
            except:    
                continue

jmpdict

{'1nnA_2nnA': [1],
 '1nnA_2nnB': [3],
 '1nnA_3nnB': [4],
 '1nnA_3nnC': [2],
 '1nnB_2nnB': [6],
 '1nnB_3nnB': [7],
 '1nnB_5nnB': [5],
 '2nnA_4nnC': [10],
 '2nnB_4nnB': [9],
 '2nnB_4nnC': [8]}

In [6]:
E_f_pdb = 4.081701163
name_to_en = {"1nnA":-2076.29355226,
"1nnB":-2076.17073296,
"2nnA":-2076.20528052,
"2nnB":-2076.17035469,
"3nnA":-2076.12268221,
"3nnB":-2076.16878248,
"3nnC":-2076.20430949,
"4nnA":-2076.11727877,
"4nnB":-2076.17973527,
"4nnC":-2076.15460157,
"5nnA":-2076.15807100,
"5nnB":-2076.12315570}

In [7]:
E_sup_pdb = -2081.44451396
E_sup_solute = -2071.87878154 
E_bulk = -2077.21734574  #E_bulk is the same as E_ref
name_to_Ef = defaultdict(float)
for (key, E_IB) in name_to_en.items():
    # get the binding energy first
    Eb = -E_IB + E_sup_pdb + E_sup_solute - E_bulk
    # Next, get the formation energy (relative to solute formation energy)
    name_to_Ef[key] = E_f_pdb - Eb
name_to_Ef["4nnD"] = E_f_pdb
name_to_Ef

defaultdict(float,
            {'1nnA': 3.8940986630003156,
             '1nnB': 4.016917963000489,
             '2nnA': 3.9823704030002975,
             '2nnB': 4.017296233000545,
             '3nnA': 4.064968713000164,
             '3nnB': 4.018868443000267,
             '3nnC': 3.983341433000371,
             '4nnA': 4.070372153000173,
             '4nnB': 4.007915653000348,
             '4nnC': 4.033049353000317,
             '5nnA': 4.029579923000549,
             '5nnB': 4.0644952230002875,
             '4nnD': 4.081701163})

In [8]:
# The complex energies are set. Now, we set the mixed dumbbell energies
E_b_mdb = 2075.72579114 + E_sup_pdb + E_sup_solute - E_bulk
E_f_mdb = E_f_pdb - E_b_mdb
E_f_mdb, E_f_mdb-E_f_pdb

(4.461859783000202, 0.38015862000020206)

In [9]:
# J_3_1nnA_2nnA   -2075.9362
# J_3_1nnA_2nnB   -2075.9725
# J_3_1nnA_3nnB   -2075.9056
# J_3_1nnA_3nnC   -2075.9349
# J_3_1nnB_2nnB   -2075.8653
# J_3_1nnB_3nnB   -2075.8672
# J_3_1nnB_5nnB   -2075.8141
# J_3_2nnA_4nnC   -2075.8528
# J_3_2nnB_4nnB   -2075.9148
# J_3_2nnB_4nnC   -2075.8685

Jname_2_TS_en = {"1nnA_2nnA": -2075.9362, "1nnA_2nnB": -2075.9725, "1nnA_3nnB": -2075.9056,
                 "1nnA_3nnC": -2075.9349, "1nnB_2nnB": -2075.8653, "1nnB_3nnB": -2075.8672,
                 "1nnB_5nnB": -2075.8141, "2nnA_4nnC": -2075.8528, "2nnB_4nnB": -2075.9148, 
                 "2nnB_4nnC": -2075.8685}

In [10]:
# Now, we have to find the TS energies.
Jname_2_ef_ts = defaultdict(float)
for (key, E_IB) in Jname_2_TS_en.items():
    Eb = -E_IB + E_sup_pdb + E_sup_solute - E_bulk
    # Next, get the formation energy (relative to solute formation energy)
    Jname_2_ef_ts[key] = E_f_pdb - Eb

In [11]:
Jname_2_mig = defaultdict(float)
for (key, TS_en) in Jname_2_ef_ts.items():
    initstar = key[:4]
    Jname_2_mig[key] = TS_en - name_to_Ef[initstar]
Jname_2_mig

defaultdict(float,
            {'1nnA_2nnA': 0.35735225999997056,
             '1nnA_2nnB': 0.32105226000021503,
             '1nnA_3nnB': 0.38795226000002003,
             '1nnA_3nnC': 0.3586522599998716,
             '1nnB_2nnB': 0.3054329599999619,
             '1nnB_3nnB': 0.3035329599997567,
             '1nnB_5nnB': 0.35663295999984257,
             '2nnA_4nnC': 0.3524805199999719,
             '2nnB_4nnB': 0.2555546899998262,
             '2nnB_4nnC': 0.30185468999980003})

In [12]:
# omega2 and omega43 Johnson jumps
E_IB_43, E_IB_2 = -2075.72579114, -2075.3619
Eb_43, Eb_2 = -E_IB_43 + E_sup_pdb + E_sup_solute - E_bulk, -E_IB_2 + E_sup_pdb + E_sup_solute - E_bulk 
# Next, get the formation energy (relative to solute formation energy)
ef_ts_43 = E_f_pdb - Eb_43
ef_ts_2 = E_f_pdb - Eb_2
print(ef_ts_2, ef_ts_43)
print(ef_ts_2-E_f_mdb, ef_ts_43 - name_to_Ef["1nnB"])

4.825750923000507 4.461859783000202
0.3638911400003053 0.4449418199997126


In [13]:
# Get the temperatures
temp = []
with open("../SCMF/PDC_ratio_Cu.dat", "r") as fl:
    for line in fl:
        arr = line.split()
        temp.append(float(arr[0]))
temp = np.array(temp)

## Cu calculations

In [14]:
vu0 = 4.4447
vu2 = 2.6848
Dconv=1e-2
# Need to change the way we are dealing with mdbs to be able to change the pre-factors with consistent results.

predb0, enedb0 = np.ones(1)*np.exp(0.05), np.array([E_f_pdb])

# We'll measure every formation energy relative to the solute formation energy.
preS, eneS = np.ones(1), np.array([0.0])

# Next, interaction or the excess energies and pre-factors for solutes and dumbbells.
preSdb, eneSdb = np.ones(onsagercalculator.thermo.mixedstartindex), \
                 np.zeros(onsagercalculator.thermo.mixedstartindex)
# Now, we go over the necessary stars and assign interaction energies
for (key, index) in name_to_themo_star.items():
    eneSdb[index] = name_to_Ef[key] - E_f_pdb

predb2, enedb2 = np.ones(1), np.array([E_f_mdb])

# Transition state energies - For omega0, omega2 and omega43, the first type is the Johnson jump,
# and the second one is the Rigid jump, the third one the on-site rotation.

# Omega0 TS eneriges
preT0, eneT0 = Dconv*vu0*np.ones(1), np.array([E_f_pdb+0.335115123])

# Omega2 TS energies
Nj2 = len(onsagercalculator.jnet2)
preT2, eneT2 = Dconv*vu2*np.ones(Nj2), np.array([ef_ts_2])

# Omega43 TS energies
preT43, eneT43 = Dconv*vu0*np.ones(1), np.array([ef_ts_43])

# Omega1 TS energies - need to be careful here
preT1 = Dconv*vu0*np.ones(len(onsagercalculator.jnet1))
eneT1 = np.array([eneT0[i] for i in onsagercalculator.om1types])
# Now, we go over the jumps that are provided and make the necessary changes
for (key, index) in jmpdict.items():
    eneT1[index] = Jname_2_ef_ts[key]

In [15]:
# Then we calculate the transport coefficients
from tqdm import tqdm

diff_aa_Cu = np.zeros(len(temp))
diff_ab_Cu = np.zeros(len(temp))
diff_bb = np.zeros(len(temp))
diff_bb_non_loc = np.zeros(len(temp))

start = time.time()
for i in tqdm(range(len(temp)), position=0, leave=True):
    T = temp[i]
    kT = kB*T
    bFdb0, bFdb2, bFS, bFSdb, bFT0, bFT1, bFT2, bFT3, bFT4 = \
        onsagercalculator.preene2betafree(kT, predb0, enedb0, preS, eneS, preSdb, eneSdb, predb2, enedb2,
                                               preT0, eneT0, preT2, eneT2, preT1, eneT1, preT43, eneT43)

    # get the probabilities and other data from L_ij
    L0bb, (L_uc_aa,L_c_aa), (L_uc_bb,L_c_bb), (L_uc_ab,L_c_ab)=\
    onsagercalculator.L_ij(bFdb0, bFT0, bFdb2, bFT2, bFS, bFSdb, bFT1, bFT3, bFT4)
    
    L_aa = L_uc_aa + L_c_aa
    L_bb = L_uc_bb + L_c_bb
    L_ab = L_uc_ab + L_c_ab
    
    diff_aa_Cu[i] = L_aa[0][0]
    diff_ab_Cu[i] = L_ab[0][0]
    diff_bb[i] = L_bb[0][0]
    diff_bb_non_loc[i] = L0bb[0][0]
        
print(time.time() - start)

100%|██████████| 381/381 [26:03<00:00,  4.10s/it]

1563.1436440944672


In [20]:
# Now save the data
import h5py
with h5py.File("Cu_data_60RT.h5","w") as fl:
    fl.create_dataset("diff_aa", data=diff_aa_Cu)
    fl.create_dataset("diff_ab", data=diff_ab_Cu)
    fl.create_dataset("diff_bb_nl", data=diff_bb_non_loc)
    fl.create_dataset("diff_bb", data=diff_bb)
    fl.create_dataset("Temp", data=np.array(temp))

In [22]:
# Now let's do the infinite temeperature limit
kT = np.inf
bFdb0, bFdb2, bFS, bFSdb, bFT0, bFT1, bFT2, bFT3, bFT4 = \
    onsagercalculator.preene2betafree(kT, predb0, enedb0, preS, eneS, preSdb, eneSdb, predb2, enedb2,
                                           preT0, eneT0, preT2, eneT2, preT1, eneT1, preT43, eneT43)
#     bFdicts[i] = [bFdb0, bFdb2, bFS, bFSdb, bFT0, bFT1, bFT2, bFT3, bFT4]
# get the probabilities and other data from L_ij
L0bb, (L_uc_aa,L_c_aa), (L_uc_bb,L_c_bb), (L_uc_ab,L_c_ab)=\
onsagercalculator.L_ij(bFdb0, bFT0, bFdb2, bFT2, bFS, bFSdb, bFT1, bFT3, bFT4)

L_aa = L_uc_aa + L_c_aa
L_bb = L_uc_bb + L_c_bb
L_ab = L_uc_ab + L_c_ab

In [23]:
L_ab[0][0]/L_aa[0][0]

2.5312788703457465